In [2]:
import torch
import os
from torchvision.utils import save_image
from tqdm import tqdm
import torch.nn as nn
import numpy as np

In [3]:
# --- Set device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
# --- Generator Definition ---
# Generator Model
class Generator(nn.Module):
    def __init__(self, latent_dim, img_channels=1, feature_g=64):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            # latent_dim x 1 x 1 → 4x4
            nn.ConvTranspose2d(latent_dim, feature_g * 16, 4, 1, 0, bias=False),
            nn.BatchNorm2d(feature_g * 16),
            nn.ReLU(True),
        
            # 4x4 → 8x8
            nn.ConvTranspose2d(feature_g * 16, feature_g * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_g * 8),
            nn.ReLU(True),
        
            # 8x8 → 16x16
            nn.ConvTranspose2d(feature_g * 8, feature_g * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_g * 4),
            nn.ReLU(True),
        
            # 16x16 → 32x32
            nn.ConvTranspose2d(feature_g * 4, feature_g * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_g * 2),
            nn.ReLU(True),
        
            # 32x32 → 64x64
            nn.ConvTranspose2d(feature_g * 2, feature_g, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_g),
            nn.ReLU(True),
        
            # 64x64 → 128x128
            nn.ConvTranspose2d(feature_g, feature_g // 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_g // 2),
            nn.ReLU(True),
        
            # 128x128 → 256x256
            nn.ConvTranspose2d(feature_g // 2, img_channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )


    def forward(self, x):
        return self.gen(x)

In [5]:
# --- Config ---
latent_dim = 100
num_images = 12760  # total images you want
batch_size = 64     # batch size for speed
output_dir = "generated_fakes_unique"
os.makedirs(output_dir, exist_ok=True)

In [6]:
# --- Load model ---
generator = Generator(latent_dim=latent_dim).to(device)
generator.load_state_dict(torch.load("checkpoints/generator_epoch_100.pth", map_location=device))
generator.eval()


Generator(
  (gen): Sequential(
    (0): ConvTranspose2d(100, 1024, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): BatchNo

In [7]:
# --- Track used latent vectors ---
used_latents = set()

def generate_unique_noise(batch_size, latent_dim):
    noise_list = []
    while len(noise_list) < batch_size:
        new_noise = torch.randn(1, latent_dim, 1, 1)
        noise_tuple = tuple(new_noise.view(-1).round(decimals=4).tolist())
        if noise_tuple not in used_latents:
            used_latents.add(noise_tuple)
            noise_list.append(new_noise)
    return torch.cat(noise_list, dim=0)

In [8]:
# --- Generate and save images ---
with torch.no_grad():
    current_index = 0
    with tqdm(total=num_images, desc="Generating Unique Images") as pbar:
        while current_index < num_images:
            curr_batch_size = min(batch_size, num_images - current_index)
            noise = generate_unique_noise(curr_batch_size, latent_dim).to(device)
            fake_imgs = generator(noise)

            for i, img in enumerate(fake_imgs):
                filename = os.path.join(output_dir, f"fake_{current_index + i:05d}.png")
                save_image(img, filename, normalize=True)
            current_index += curr_batch_size
            pbar.update(curr_batch_size)

print(f"✅ {num_images} unique images saved in '{output_dir}'")

Generating Unique Images: 100%|██████████████████████████████████████████████████| 12760/12760 [12:40<00:00, 16.78it/s]

✅ 12760 unique images saved in 'generated_fakes_unique'
